In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize
import scipy.special
def neglogverosimilitud(theta, *args):
    X, y = args
    loglambda = np.dot(X, theta) 
    return -np.sum(y*loglambda - np.exp(loglambda) - scipy.special.loggamma(y+1)) 
def grad_neglogverosimilitud(theta, *args):
    X, y = args
    loglambda = np.dot(X, theta)    
    error = y - np.exp(loglambda)
    return -np.sum(error.reshape(-1,1)*X, axis=0)
def pseudo_r2(theta, *args):
    X, y = args
    best_theta0 = np.mean(y)
    logL0 = -neglogverosimilitud(np.array([best_theta0, 0., 0., 0.]), X, y)
    return (logL0 + neglogverosimilitud(theta, X, y))/logL0
def train_model(X, y):
    res = scipy.optimize.minimize(fun=neglogverosimilitud, x0=0.1*np.random.randn(4), 
                                  method='BFGS', jac=grad_neglogverosimilitud,
                                  args=(X, y), tol=1e-2)
    return res.x
def bootstrap(X, y, T):
    N = len(y)
    theta_boot = np.zeros(shape=(T, 4))
    pr2_boot = np.zeros(shape=(T,))
    for t in range(T):
        idx = np.random.choice(N, N, replace=True)
        theta_boot[t, :] = train_model(X[idx, :], y[idx])
        pr2_boot[t] = pseudo_r2(0.1*np.random.randn(4), X[idx, :], y[idx])
    best_theta = train_model(X, y)
    best_pr2 = pseudo_r2(best_theta, X, y)
    def plot_bootstrap(ax, dist, best, title):
        pp = np.percentile(dist, [5, 95])
        ax.hist(dist)
        ax.axvline(best, ls='--', c='r')
        ax.axvline(pp[0], ls='--', c='k')
        ax.axvline(pp[1], ls='--', c='k')
        ax.set_title(title)   
    fig, ax = plt.subplots(2, 2, figsize=(6, 6), tight_layout=True)
    for k, (ax_, theta) in enumerate(zip(ax.ravel(), theta_boot.T)):
        plot_bootstrap(ax_, theta, best_theta[k], f"theta{k}")
    fig, ax = plt.subplots(figsize=(3, 3), tight_layout=True)
    plot_bootstrap(ax, pr2_boot, best_pr2, "pseudo R2")
y = df["nbillonarios"].values
names = df.index.values
ordering = np.argsort(y)[::-1]
y = y[ordering]
names = names[ordering]
X = df[["logpibpc", "logpob", "gatt"]].values[ordering, :]
X = (X - np.mean(X, axis=0, keepdims=True))/np.std(X, axis=0, keepdims=True)
X = np.concatenate((np.ones(shape=(len(y), 1)), X), axis=1)
bootstrap(X, y, T=100)
best_theta = train_model(X, y)
yhat = np.exp(np.dot(X, best_theta))
fig, ax = plt.subplots(figsize=(6, 8), tight_layout=True)
ax.barh(range(10), y[:10], alpha=0.5, label='True')
ax.barh(range(10), yhat[:10], alpha=0.5, label='Expected')
plt.legend()
ax.set_yticks(range(10))
ax.set_yticklabels(names[:10]);